In [1]:
%cd ../
!pwd 

/home/timur.bikbulatov/personal/aa_on_vad
/home/timur.bikbulatov/personal/aa_on_vad


/home/timur.bikbulatov/miniconda3/envs/aaml/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import plotly.graph_objects as go
import numpy as np

def plot(y:list):
    trace = []
    colors = [
        'Blue',
        'Orange',
        'Green',
        'Red',
        'Purple',
        'Magenta',
        'Cyan',
        'Brown',
        'Pink',
        'Lime',
        'Yellow',
        'Teal',
        'Olive',
        'Navy',
        'Maroon',
        'Coral',
        'Gold',
        'Indigo',
        'Turquoise',
        'Lavender',
        'Mint',
        'Silver',
        ]
    for ik, y_ in enumerate(y):
        trace.append(go.Scatter(x=np.arange(len(y_)), y=y_, mode='lines', name=f'arg # {ik + 1}', line=dict(color=colors[ik])))

    # Combining both traces into one figure
    fig = go.Figure(data=trace)

    # Setting the layout
    fig.update_layout(
        title='Two Line Charts on One Plot',
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        showlegend=True
    )

    # Display the plot
    fig.show()

In [3]:
import IPython
from functools import partial
play = partial(IPython.display.Audio,
               rate=16000)

In [4]:
import torch

In [5]:
torch.set_num_threads(1)

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
(get_speech_timestamps, _, read_audio, _, _) = utils

Using cache found in /home/timur.bikbulatov/.cache/torch/hub/snakers4_silero-vad_master


In [6]:
wav = read_audio('datasets/speech16.wav')
speech_timestamps = get_speech_timestamps(
  wav,
  model,
  return_seconds=True,  # Return speech timestamps in seconds (default is samples)
)

print(speech_timestamps)

[{'start': 0.3, 'end': 2.1}, {'start': 3.0, 'end': 5.2}, {'start': 6.7, 'end': 7.3}, {'start': 7.5, 'end': 8.2}, {'start': 12.0, 'end': 15.9}, {'start': 17.2, 'end': 20.4}, {'start': 20.6, 'end': 23.4}, {'start': 23.9, 'end': 24.6}, {'start': 24.9, 'end': 28.5}, {'start': 28.9, 'end': 34.9}, {'start': 35.1, 'end': 35.8}, {'start': 36.0, 'end': 42.5}, {'start': 42.9, 'end': 44.3}, {'start': 44.5, 'end': 54.3}, {'start': 54.4, 'end': 56.6}, {'start': 56.7, 'end': 57.2}, {'start': 57.3, 'end': 61.8}, {'start': 61.9, 'end': 65.2}, {'start': 65.7, 'end': 67.6}, {'start': 68.5, 'end': 70.4}, {'start': 73.9, 'end': 78.6}, {'start': 79.4, 'end': 84.3}]


In [7]:
wav1 = torch.tensor(wav, requires_grad=True)
wav1

/tmp/ipykernel_2353896/1298232958.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  wav1 = torch.tensor(wav, requires_grad=True)


tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)

In [16]:
o1 = model._model.stft(wav1.reshape(1, -1))
o1

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]],
       grad_fn=<DifferentiableGraphBackward>)

In [19]:
o2 = model._model.encoder(o1)
o2

tensor([[[0.2081, 0.0000, 0.0032,  ..., 0.0000, 0.0000, 0.5323],
         [0.0697, 1.6500, 1.6210,  ..., 1.5718, 1.4964, 0.0000],
         [0.2396, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9742],
         ...,
         [0.3942, 1.0355, 1.2398,  ..., 1.0459, 1.0875, 1.0967],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       grad_fn=<ReluBackward0>)

In [34]:
o2.squeeze(-1).size()

torch.Size([1, 128, 2731])

In [25]:
state = torch.zeros(0)

In [31]:
o3, state = model._model.decoder(o2[0,:, 0], state)
o3

tensor([[0.0123]], grad_fn=<SigmoidBackward0>)

In [28]:
o2.size()

torch.Size([1, 128, 2731])

In [8]:
import torchaudio

def process_audio_internal(model, waveform, sr=16000):
    chunk_size = 512
    
    context_size = 32
    total_pad = chunk_size - (waveform.shape[1] % chunk_size)
    padded_waveform = torch.nn.functional.pad(waveform, (context_size, total_pad + context_size))
    stft_features = model._model.stft(padded_waveform)
    
    encoder_features = model._model.encoder(stft_features)
    
    state = torch.zeros(0)
    
    predictions = []
    print(f"STFT feature shape: {stft_features.shape}")
    print(f"Encoder feature shape: {encoder_features.shape}")
    for i in range(encoder_features.shape[2]):
        current_features = encoder_features[:, :, i:i+1]
        
        pred, state = model._model.decoder(current_features, state)
        predictions.append(pred)
    
    all_predictions = torch.cat(predictions, dim=2)
    valid_predictions = all_predictions[:, :, context_size//chunk_size:-context_size//chunk_size]
    
    return valid_predictions

In [26]:
def process_file(audio_path):
    waveform, source_sr = torchaudio.load(audio_path)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    
    if source_sr != 16000:
        resampler = torchaudio.transforms.Resample(source_sr, 16000)
        waveform = resampler(waveform)
    
    predictions = process_audio_internal(model, waveform)
    
    speech_mask = (predictions[0, 0] > 0.3).cpu().numpy()
    
    
    print(f"Final predictions shape: {predictions.shape}")
    print(f"Number of speech windows detected: {speech_mask.sum()}")
    
    return predictions, speech_mask

In [27]:
predictions, speech_mask = process_file("datasets/speech16.wav")
wav = read_audio('datasets/speech16.wav')

STFT feature shape: torch.Size([1, 129, 10924])
Encoder feature shape: torch.Size([1, 128, 2731])
Final predictions shape: torch.Size([1, 1, 2730])
Number of speech windows detected: 943


In [35]:
predictions

tensor([[[0.0123, 0.0085, 0.0177,  ..., 0.0155, 0.0154, 0.0153]]],
       grad_fn=<SliceBackward0>)

In [28]:
speech_timestamps

[{'start': 0.3, 'end': 2.1},
 {'start': 3.0, 'end': 5.2},
 {'start': 6.7, 'end': 7.3},
 {'start': 7.5, 'end': 8.2},
 {'start': 12.0, 'end': 15.9},
 {'start': 17.2, 'end': 20.4},
 {'start': 20.6, 'end': 23.4},
 {'start': 23.9, 'end': 24.6},
 {'start': 24.9, 'end': 28.5},
 {'start': 28.9, 'end': 34.9},
 {'start': 35.1, 'end': 35.8},
 {'start': 36.0, 'end': 42.5},
 {'start': 42.9, 'end': 44.3},
 {'start': 44.5, 'end': 54.3},
 {'start': 54.4, 'end': 56.6},
 {'start': 56.7, 'end': 57.2},
 {'start': 57.3, 'end': 61.8},
 {'start': 61.9, 'end': 65.2},
 {'start': 65.7, 'end': 67.6},
 {'start': 68.5, 'end': 70.4},
 {'start': 73.9, 'end': 78.6},
 {'start': 79.4, 'end': 84.3}]

In [29]:
timestamps = np.zeros(wav.size()[0])

for t in speech_timestamps:
    timestamps[int(t['start']*16000):int(t['end']*16000)] = 1

In [30]:
np.array(speech_mask).shape

(2730,)

In [31]:
wav.size()[-1] / 2730

512.0978021978023

In [32]:
preds = np.repeat(predictions.detach().numpy(), 512)
smask = np.repeat(np.array(speech_mask), 512)

In [ ]:
plot([wav.numpy()[:300000],smask[:300000].astype(int), timestamps[:300000], preds[:300000]])

In [60]:
predictions, speech_mask = process_file("datasets/woaini.wav")
preds = np.repeat(predictions.detach().numpy(), 512)

STFT feature shape: torch.Size([1, 129, 11988])
Encoder feature shape: torch.Size([1, 128, 2997])
Final predictions shape: torch.Size([1, 1, 2996])
Number of speech windows detected: 0


In [62]:
wav = read_audio('datasets/woaini.wav')
wav.size()


torch.Size([1534375])

In [ ]:
plot([wav[:300000], preds[:300000]])

In [64]:
import soundfile as sf

w, sr = sf.read('datasets/woaini.wav')

In [65]:
sr

44100

In [66]:
w.shape

(4229120, 2)

In [67]:
4229120 / 44100 * 16000

1534374.603174603

In [ ]:
import warnings

@torch.no_grad()
def get_speech_timestamps(audio: torch.Tensor,
                          model,
                          threshold: float = 0.5,
                          sampling_rate: int = 16000,
                          min_speech_duration_ms: int = 250,
                          max_speech_duration_s: float = float('inf'),
                          min_silence_duration_ms: int = 100,
                          speech_pad_ms: int = 30,
                          return_seconds: bool = False,
                          visualize_probs: bool = False,
                          neg_threshold: float = None,
                          window_size_samples: int = 512,):

    window_size_samples = 512 if sampling_rate == 16000 else 256

    # model.reset_states()
    min_speech_samples = sampling_rate * min_speech_duration_ms / 1000
    speech_pad_samples = sampling_rate * speech_pad_ms / 1000
    max_speech_samples = sampling_rate * max_speech_duration_s - window_size_samples - 2 * speech_pad_samples
    min_silence_samples = sampling_rate * min_silence_duration_ms / 1000
    min_silence_samples_at_max_speech = sampling_rate * 98 / 1000

    audio_length_samples = len(audio)
    state = torch.zeros(0)
    speech_probs = []
    for current_start_sample in range(0, audio_length_samples, window_size_samples):
        chunk = audio[current_start_sample: current_start_sample + window_size_samples]
        if len(chunk) < window_size_samples:
            chunk = torch.nn.functional.pad(chunk, (0, int(window_size_samples - len(chunk))))
        # speech_prob = model(chunk, sampling_rate).item()
        stft_features = model._model.stft(chunk)

        speech_probs.append(speech_prob)

    return speech_prob


In [ ]:
chunk_size = 512
    
    context_size = 32
    total_pad = chunk_size - (waveform.shape[1] % chunk_size)
    padded_waveform = torch.nn.functional.pad(waveform, (context_size, total_pad + context_size))
    stft_features = model._model.stft(padded_waveform)
    
    encoder_features = model._model.encoder(stft_features)
    
    state = torch.zeros(0)
    
    predictions = []
    print(f"STFT feature shape: {stft_features.shape}")
    print(f"Encoder feature shape: {encoder_features.shape}")
    for i in range(encoder_features.shape[2]):
        current_features = encoder_features[:, :, i:i+1]
        
        pred, state = model._model.decoder(current_features, state)
        predictions.append(pred)
    
    all_predictions = torch.cat(predictions, dim=2)
    valid_predictions = all_predictions[:, :, context_size//chunk_size:-context_size//chunk_size]
    
    return valid_predictions

In [77]:
model.reset_states()
stft = model._model.stft(wav[:512].reshape(1,-1))
encoder_out = model._model.encoder(stft)
state, output = model._model.decoder(encoder_out, torch.zeros(0))
model._model.decoder.decoder(output[0].unsqueeze(-1))

torch.Size([2, 1, 128])

In [85]:
model._model.decoder.rnn

RecursiveScriptModule(original_name=LSTMCell)

In [84]:
model.reset_states()
stft = model._model.stft(wav[:512].reshape(1,-1))
encoder_out = model._model.encoder(stft)
h = c = torch.zeros(encoder_out.size(0), 128)  # Hidden and cell states for LSTM
decoder_input = encoder_out.squeeze(-1)
x = h.unsqueeze(-1).float()  # Shape: [batch_size, features, 1]
out = decoder.decoder(x)  # Shape: [batch_size, 1, 1]


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [79]:
model._model.reset_states()

AttributeError: 'RecursiveScriptModule' object has no attribute 'reset_states'

In [72]:
output.reshape(2,-1).unsqueeze(-1).shape

torch.Size([2, 128, 1])

In [64]:
model.reset_states()
model(wav[:512].reshape(1,-1), 16000)

tensor([[0.0120]], grad_fn=<UnsqueezeBackward0>)

In [86]:
# Prepare the audio input
padded_waveform = wav.reshape(1, -1)  # Shape: [batch_size, time]

# Extract STFT features
stft_features = model._model.stft(padded_waveform)
print("stft_features shape:", stft_features.shape)

# Pass through the encoder
encoder_out = model._model.encoder(stft_features)
print("encoder_out shape:", encoder_out.shape)

# Initialize decoder state
decoder = model._model.decoder
state = (torch.zeros(encoder_out.size(0), decoder.hidden_size),
         torch.zeros(encoder_out.size(0), decoder.hidden_size))
print("Decoder hidden size:", decoder.hidden_size)

# Process through the decoder
predictions = []
for t in range(encoder_out.size(2)):
    input_t = encoder_out[:, :, t]
    print("input_t shape:", input_t.shape)
    h, c = decoder.rnn(input_t, state)
    state = (h, c)
    x = h.unsqueeze(-1)  # Shape: [batch_size, hidden_size, 1]
    print("x shape:", x.shape)
    out = decoder.decoder(x)  # Apply the decoder layers
    print("out shape:", out.shape)
    predictions.append(out)

# Concatenate predictions
predictions = torch.cat(predictions, dim=2)
print("predictions shape:", predictions.shape)

# Get the speech probability for the first time step
speech_prob = predictions[0, 0, 0].item()
print("speech_prob:", speech_prob)

stft_features shape: torch.Size([1, 129, 10921])
encoder_out shape: torch.Size([1, 128, 2731])


AttributeError: 'RecursiveScriptModule' object has no attribute 'hidden_size'

In [87]:
decoder

RecursiveScriptModule(
  original_name=VADDecoderRNNJIT
  (rnn): RecursiveScriptModule(original_name=LSTMCell)
  (decoder): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Dropout)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Conv1d)
    (3): RecursiveScriptModule(original_name=Sigmoid)
  )
)